# Importing Library files

* pandas was required for reading the csv file
* numpy was required for array manupulation
* Pipeline is used to make the model easily deployable


In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline

Reading the CSV file where the data is saved

In [3]:
df=pd.read_csv('reddit.csv')

Incase of any missing data fillna was used to remove error
later a new data column text was made by combining all the columns, this made feeding data into the classifier easier as a single columns is needed to be vectorized and fitted

In [4]:
df.body.fillna(' ',inplace=True)
df.comments.fillna(' ',inplace=True)
df.title.fillna(' ',inplace=True)
df.flair.fillna(' ',inplace=True)
df=df.sample(frac=1)
df['text'] = df['title'].astype(str)+' ' + df['body'].astype(str) + " " + df['comments'].astype(str)

The lemma of each words is checked and the stopwords or punctuations are removed.
The 'en' package was able to provide hte highest accuracy, and hence it was used

In [11]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
nlp = spacy.load('en')
def lemmatize(msg):
    
    doc = nlp(msg)
    res=[]
    
    for token in doc:
        if(token.is_stop or token.is_punct or not(token.is_oov)): #Removing stopwords punctuations and words out of vocab
            pass
        else:
            res.append(token.lemma_.lower())
    
    return " ".join(res)
df['text'] = df['text'].apply(lemmatize)

The text colummn is divided inton train and test subsets

In [6]:
X=np.array(df['text'])
y = df["flair"]
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

The pipeline is fiited is defined, RandomForest provided the highest accuracy and hence it was used

In [7]:

model= Pipeline([('preprocessin',TfidfVectorizer()),
                ('classifier',RandomForestClassifier(n_estimators=210,n_jobs=-1,random_state=0))])

The pipeline is trained according to test data

In [8]:
model.fit(X_train,y_train)

Pipeline(steps=[('preprocessin', TfidfVectorizer()),
                ('classifier',
                 RandomForestClassifier(n_estimators=210, n_jobs=-1,
                                        random_state=0))])

Score is Checked

In [9]:
model.score(X_test,y_test)

0.925

Finally the model is saved

In [10]:
import pickle
model.fit(X,y)
filename='model.pkl'
pickle.dump(model,open(filename,'wb'))

In [10]:
clff=pickle.load(open('model.pkl','rb'))
